In [19]:
import pandas as pd
from g4f.client import Client
import re

In [20]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 JSON 文件並轉換為 DataFrame。
    """
    try:
        import json
        with open(file_path, "r", encoding="utf-8") as file:
            raw_data = json.load(file)
        return pd.DataFrame(raw_data)
    except json.JSONDecodeError as e:
        raise Exception(f"JSON 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")


In [21]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [22]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。"
    )
    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [23]:

def load_data(file_path: str) -> pd.DataFrame:
    """
    JSON 文件並轉換為 DataFrame。
    """
    try:
        import json
        with open(file_path, "r", encoding="utf-8") as file:
            raw_data = json.load(file)
        return pd.DataFrame(raw_data)
    except json.JSONDecodeError as e:
        raise Exception(f"JSON 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")


In [25]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[  
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"


In [28]:
if __name__ == "__main__":
    # 示例使用
    file_path = file_path = r"C:\Project_Tibame_Capstone\Project\flask_app\LLM4report\json.ingredient\tsmc_operating_profit_margin_2010_2023.json"
    user_input = "請分析數據並提供詳細的見解和建議，並提供報告。"
    try:
        report = generate_report(file_path, user_input)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

{'chatId': '2fe3bc35-3b76-400f-80e9-bef63a252e76', 'frequency_penalty': 0, 'max_tokens': 4000, 'messages': [{'role': 'system', 'content': '你是一位專業的數據科學家，擅長分析數據並從中提取見解。以下是一些統計數據，請根據這些數據提供深入分析和建議：\n1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\n5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。'}, {'role': 'user', 'content': '以下是數據的描述性統計結果：\n               年份         季度     營業利潤率\ncount    56.00000  56.000000  56.00000\nmean   2016.50000   2.500000  36.40000\nstd       4.06761   1.128152   0.69544\nmin    2010.00000   1.000000  35.00000\n25%    2013.00000   1.750000  35.90000\n50%    2016.50000   2.500000  36.40000\n75%    2020.00000   3.250000  36.90000\nmax    2023.00000   4.000000  37.80000\n請根據這些數據回答以下問題：請分析數據並提供詳細的見解和建議，並提供報告。'}], 'model': 'gpt-4o-mini', 'personaId': 'gemini-1-5-flash', 'presence_penalty': 0, 'stream': False, 'temperature': 0.1, 'top_p': 0.95}
{'chatId': 'e67dfd5d-0f3d-4071-a919-4454f923fd48',